# 05 · CNN Baseline (Real Negatives)
**Purpose** This notebook revisits the baseline 3D CNN but replaces the previous synthetic noise negatives with real background patches extracted from patient scans (far from annotated nodules). This makes the task more realistic and provides a stronger, fairer baseline.

Key steps include:
- Building real negative samples per scan using distance constraints from nodule centers
- Recreating grouped train/validation splits (by seriesuid) to prevent leakage
- Training the same lightweight 3D CNN architecture for a like-for-like comparison 
- Evaluating with AUROC and threshold metrics, and comparing against the synthetic-negatives baseline

This iteration aims to reduce optimism in the baseline and better reflect true deployment conditions.

In this cell we are going to import the required libraries, set the compute device, define paths for the positive (nodule) and real negative background patches, load their indices (patch_index.csv and bg_index.csv) while assigning labels (1 for positives, 0 for negatives), and print the resulting DataFrame shapes to confirm the data has been mounted correctly.

In [ ]:
import random
import torch
import pandas as pd
import numpy as np 
import torch.nn as nn
import torchmetrics as tm

from pathlib import Path
from torch.utils.data import Dataset, DataLoader
from torchvision.models.video import r3d_18
from sklearn.model_selection import GroupShuffleSplit
from tqdm.auto import tqdm

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

POS_DATA = Path("/kaggle/input/patches")                               # patches_64mm + patch_index.csv
NEG_DATA = Path("/kaggle/input/negative-mining-output-data")           # bg_patches_64mm + bg_index.csv

POS_DIR = POS_DATA/"patches_64mm"
NEG_DIR = NEG_DATA/"bg_patches_64mm"

pos_df = pd.read_csv(POS_DATA/"patch_index.csv").assign(label=1)
neg_df = pd.read_csv(NEG_DATA/"bg_index.csv").assign(label=0)

print(pos_df.shape, neg_df.shape)

(1186, 7) (601, 8)


In this cell we are going to create a grouped train/validation split using GroupShuffleSplit on positives by seriesuid (to avoid patient-level leakage), then align real negatives to each split by filtering neg_df to the same seriesuid sets. Finally, we print the shapes to verify that positives and negatives are correctly partitioned for both train and validation.

In [2]:
groups = pos_df['seriesuid'].values
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
tr_idx, va_idx = next(gss.split(pos_df, groups=groups))
pos_tr, pos_va = pos_df.iloc[tr_idx].reset_index(drop=True), pos_df.iloc[va_idx].reset_index(drop=True)

# pair negatives to the same split by seriesuid
neg_tr = neg_df[neg_df.seriesuid.isin(pos_tr.seriesuid.unique())].reset_index(drop=True)
neg_va = neg_df[neg_df.seriesuid.isin(pos_va.seriesuid.unique())].reset_index(drop=True)

print(pos_tr.shape, neg_tr.shape, "|", pos_va.shape, neg_va.shape)

(938, 7) (480, 8) | (248, 7) (121, 8)


In this cell we are going to define a dataset RealNegDS that loads nodule patches from POS_DIR and real background patches from NEG_DIR, normalizes shape to 64³, and applies light augmentation to positives. We then instantiate train/val datasets and wrap them in DataLoaders (shuffled train, non-shuffled val).

In [3]:
def to_64(c):
    if c.shape == (64,64,64): return c.astype(np.float32, copy=False)
    # centre-crop/pad per axis
    def fix(a, ax, tgt=64):
        s=a.shape[ax]
        if s>=tgt:
            st=(s-tgt)//2; sl=[slice(None)]*3; sl[ax]=slice(st,st+tgt); return a[tuple(sl)]
        b=(tgt-s)//2; a2=tgt-s-b; pad=[(0,0)]*3; pad[ax]=(b,a2); return np.pad(a,pad)
    c=fix(fix(fix(c,0),1),2); return c.astype(np.float32, copy=False)

class RealNegDS(Dataset):
    def __init__(self, pos, neg, pos_dir, neg_dir, augment=True):
        self.df = pd.concat([pos, neg]).sample(frac=1, random_state=0).reset_index(drop=True)
        self.pos_dir, self.neg_dir, self.aug = pos_dir, neg_dir, augment
    def __len__(self): return len(self.df)
    def __getitem__(self, i):
        r = self.df.iloc[i]
        pth = (self.pos_dir if r.label==1 else self.neg_dir) / r.patch_file
        cube = np.load(pth); cube = to_64(cube)
        if self.aug and r.label==1:
            if random.random()<.5: cube = cube[::-1]
            if random.random()<.5: cube = np.rot90(cube, 1, (1,2))
        cube = np.ascontiguousarray(cube)
        x = torch.from_numpy(cube).float().unsqueeze(0)
        y = torch.tensor(r.label, dtype=torch.float32)
        return x,y

train_ds = RealNegDS(pos_tr, neg_tr, POS_DIR, NEG_DIR, augment=True)
val_ds   = RealNegDS(pos_va, neg_va, POS_DIR, NEG_DIR, augment=False)
train_dl = DataLoader(train_ds, batch_size=16, shuffle=True,  num_workers=0, pin_memory=True)
val_dl   = DataLoader(val_ds,   batch_size=32, shuffle=False, num_workers=0, pin_memory=True)

In this cell we are going to build the 3D ResNet-18 baseline adapted to single-channel input, set up BCEWithLogitsLoss, AdamW, and AUROC, and run an 8-epoch train/validate loop. We log loss and AUROC each epoch and save a checkpoint (cnn_baseline_realnegs.pt) whenever validation AUROC improves.

In [ ]:
model = r3d_18(weights=None)
model.stem[0] = nn.Conv3d(1,64,7,2,3,bias=False)
model.fc = nn.Sequential(nn.Linear(512,128), nn.ReLU(), nn.Dropout(0.3), nn.Linear(128,1))
model.to(device)

loss_fn = nn.BCEWithLogitsLoss()
opt     = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-4)
auroc   = tm.AUROC(task="binary").to(device)

best_auc=0.0
for epoch in range(8):
    # train
    model.train(); auroc.reset(); run_loss=0.0
    for xb,yb in tqdm(train_dl, leave=False):
        xb,yb = xb.to(device), yb.to(device)
        opt.zero_grad(set_to_none=True)
        logits = model(xb).squeeze()
        loss   = loss_fn(logits, yb)
        loss.backward(); opt.step()
        run_loss += loss.item()*xb.size(0)
        auroc.update(torch.sigmoid(logits).detach(), yb)
    tr_loss = run_loss/len(train_ds); tr_auc = auroc.compute().item()
    # validate
    model.eval(); auroc.reset(); run_loss=0.0
    with torch.no_grad():
        for xb,yb in val_dl:
            xb,yb = xb.to(device), yb.to(device)
            lg = model(xb).squeeze(); l = loss_fn(lg, yb)
            run_loss += l.item()*xb.size(0)
            auroc.update(torch.sigmoid(lg), yb)
    va_loss = run_loss/len(val_ds); va_auc = auroc.compute().item()
    print(f"epoch {epoch:02d} | train loss {tr_loss:.4f} auc {tr_auc:.3f} | val loss {va_loss:.4f} auc {va_auc:.3f}")
    if va_auc>best_auc:
        best_auc=va_auc; torch.save(model.state_dict(),"cnn_baseline_realnegs.pt"); print("  ↳ saved")

  0%|          | 0/89 [00:00<?, ?it/s]

epoch 00 | train loss 0.6488 auc 0.531 | val loss 0.6390 auc 0.190
  ↳ saved


  0%|          | 0/89 [00:00<?, ?it/s]

epoch 01 | train loss 0.4748 auc 0.792 | val loss 0.7693 auc 0.818
  ↳ saved


  0%|          | 0/89 [00:00<?, ?it/s]

...

epoch 05 | train loss 0.4616 auc 0.808 | val loss 0.6343 auc 0.785


  0%|          | 0/89 [00:00<?, ?it/s]

epoch 06 | train loss 0.4253 auc 0.833 | val loss 2.5729 auc 0.821


  0%|          | 0/89 [00:00<?, ?it/s]

epoch 07 | train loss 0.4133 auc 0.834 | val loss 4.5833 auc 0.820


Conclusion. Replacing synthetic noise with real background negatives yields a more realistic and stricter baseline. You should expect AUROC to drop slightly vs. the synthetic-negatives model (harder negatives), but metrics are now more trustworthy for downstream comparisons (e.g., hard negative mining, focal loss, better augmentations). Next steps: (1) calibrate a decision threshold on the val set, (2) inspect error cases—especially false positives on difficult background, and (3) consider curriculum or hard-negative sampling to further toughen training while keeping evaluation strictly grouped by seriesuid.